In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from textwrap import wrap
import requests
import json


def get_response(question, data_context):
    url = "http://localhost:11434/api/generate"
    data = {
        "model": "llama2",
        "prompt": f"{data_context}\n\n{question}",
        "stream": False
    }
    try:
        response = requests.post(url, json=data)
        response.raise_for_status()  # Raises an error for bad status codes
        response_json = response.json()
        return response_json.get('response', 'No response found')
    except requests.RequestException as e:
        return f"Error during request: {e}"

def data_summarization_agent(data):
    data_context = data.head().to_string()
    question = "Summarize the dataset."
    return get_response(question, data_context)

def statistics_agent(data):
    data_context = data.head().to_string()
    question = "What are the key statistics of this dataset?"
    return get_response(question, data_context)
    
def questions_agent(data):
    data_context = data.head().to_string()
    question = "What are some questions that can be formulated from the dataset?"
    response = get_response(question, data_context)
    return response.split('\n')

def answer_questions_agent(data, questions):
    data_context = data.head().to_string()
    answers = []
    for question in questions:
        answer = get_response(question, data_context)
        answers.append((question, answer))
    return answers

def main_ai_coordinator(file_path):
    data = pd.read_csv(file_path)

    # Data Summarization
    summary = data_summarization_agent(data)

    # Statistics Computation
    statistics = statistics_agent(data)

    # Generate Questions
    questions = questions_agent(data)

    # Generate Answers
    answers = answer_questions_agent(data, questions)

    def add_to_pdf(c, text, y_pos, max_line_length=80):
        wrapped_text = wrap(text, max_line_length)
        for line in wrapped_text:
            if y_pos < 40:
                c.showPage()
                y_pos = 750
            c.drawString(100, y_pos, line)
            y_pos -= 15
        return y_pos - 20

    # Create PDF
    output_filename ='/Users/hammad/Desktop/myproject/output_forms/output6.pdf'  # Modify this path as needed
    c = canvas.Canvas(output_filename, pagesize=letter)
    y = 750

    # Add Data Summary to PDF
    y = add_to_pdf(c, "Data Summary:", y)
    y = add_to_pdf(c, summary, y)

    # Add Key Statistics to PDF
    y = add_to_pdf(c, "Key Statistics:", y)
    y = add_to_pdf(c, statistics, y)
        
    y = add_to_pdf(c, "Generated Answers:", y)
    for question, answer in answers:
        y = add_to_pdf(c, f"Q: {question}", y)
        y = add_to_pdf(c, f"A: {answer}", y)

    c.save()
    print(f"Output saved to {output_filename}")

# Example usage
file_path =  '/Users/hammad/Desktop/myproject/Dataset/industry.csv' # Provide your dataset path
main_ai_coordinator(file_path)


Output saved to /Users/hammad/Desktop/myproject/output_forms/output6.pdf


In [ ]:
|